In [ ]:
import yfinance as yf
import datetime as dt
from datetime import datetime
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statistics import *

# Question 1

The objective of this question is to analyse the data for different instruments, based on last year’s data, and provide your opinion in terms of their respective risk and return. 

i.Download daily closingprice data for S&P 500, FTSE 100 and Gold (SPDR) for the years 2015 to 2018. 

ii.Using suitable plots, identify whether the extracted prices are stationary. Identify a statistical test to corroborate your conclusions. 

iii.Using suitable plots, identify whether the extracted prices follow a random walk process.

iv.Why are log returns normally preferred from standard arithmetic returns? 

Answer: Benefit of using returns, versus prices, is normalization: measuring all variables in a comparable metric, thus enabling evaluation of analytic relationships amongst two or more variables despite originating from price series of unequal values. This is a requirement for many multidimensional statistical analysis and machine learning techniques. 

v.Identify the first 4 distribution moments for each index/product mentioned in part (i). Foryour calculations utilise daily log returns.

vi.Comment on the measured statistics from the perspective of risk and return. In your answer compare the results obtained. 

vii.Annualize daily return (first moment) and volatility (second moment). In your scaling process assume 252 days for the year. In your answer describe the calculations/steps performed. 

In [ ]:
Tickers = ['^GSPC', '^FTSE' , 'GLD']
date_1 = dt.date(2014, 12, 31)
date_2 = dt.date(2018, 12, 31)
price_data = yf.download(Tickers, date_1, date_2)
daily_closing = price_data['Close']
daily_closing.dropna(inplace = True)

In [ ]:
daily_closing.plot(figsize = (15,9), linewidth = 1.0, grid = True)
plt.title('Daily Closing Prices')
plt.ylabel('Dollars USD $')
plt.legend()
plt.show()
plt.savefig('Daily_Closing_Prices')

In [ ]:
daily_returns = daily_closing / daily_closing.shift(1) - 1.0
daily_returns.dropna(inplace = True)
c_returns = daily_returns.expanding().sum()

In [ ]:
daily_returns.plot(figsize = (15,9), linewidth = 1.0, grid = True)
plt.title('Daily Closing Returns')
plt.legend()
plt.show()
plt.savefig('Daily_Closing_Returns')

In [ ]:
for ticker in daily_returns.columns:
    plt.figure()
    plt.hist(daily_returns[ticker], bins = 200)
    plt.title('Histogram of ' + ticker + ' returns')
    plt.show()
    plt.savefig('Histogram_Returns_' + ticker)

In [ ]:
growth_returns = (c_returns + 1) * 100

In [ ]:
growth_returns.plot(figsize = (15,9), linewidth = 1.0, grid = True)
plt.title('Growth of the Returns')
plt.legend()
plt.show()
plt.savefig('Growth_Returns')

In [ ]:
def augmented_dickey_fuller_test(values):
    result = adfuller(values)
    if result[1] > 0.05:
        print('\nADF Statistic: %f' % result[0],' p-value: %f' % result[1], '  Data is Non Stationary\n')
    else:
        print('ADF Statistic: %f' % result[0],' p-value: %f' % result[1], '  Data is Stationary\n')

In [ ]:
print('Augmented Dickey Fuller Test:')
print('P-value > 0.05 fail to reject H0 = data is non stationary\n P-value <= 0.05 reject H0 = data is stationary\n')
for ticker in daily_closing.columns:
    augmented_dickey_fuller_test(daily_closing[ticker].values)

In [ ]:
daily_log_returns = np.log(daily_closing / daily_closing.shift(1))
daily_log_returns.dropna(inplace = True)

In [ ]:
daily_log_returns.plot(figsize = (15,9), linewidth = 1.0, grid = True)
plt.title('Daily Closing Log Returns')
plt.legend()
plt.show()
plt.savefig('Daily_Closing_Log_Returns')

In [ ]:
for ticker in daily_log_returns.columns:
    plt.figure()
    plt.hist(daily_log_returns[ticker], bins = 200)
    plt.title('Histogram of ' + ticker + ' daily log returns')
    plt.show()
    plt.savefig('Histogram_Returns_Log_' + ticker)

In [ ]:
for i in range(1,5):
    print('Moment', i)
    moment = (daily_log_returns**i).mean()
    print(moment)

In [ ]:
annual_return = daily_log_returns.mean()*252
print('Annualized Daily Log returns :\n', annual_return)
annual_volatility = daily_log_returns.std()*np.sqrt(252)
print('\nAnnualized Daily Log volatility :\n', annual_volatility)

# Question 2:

Under  the  assumption of  Markov  property,  the  Binomial tree  provides  a method of  how  the  future price of an instrument, can be modelled. 

By investigating the statistics of Google daily returns for the 
period 2013 to end of 2017, construct a binomial tree model that projects Google stock pricefor the first 6 months of 2018, on a monthly basis.

i.Present the projected probability and stock price binomial trees.

ii.Utilizing  the  binomial  tree  from  (i),  what  is  the  probability  that at  the  end  of  the 6  month period the price will be greater than the starting price.

iii.Calculate the expected stock price, on a monthly basis.In your answers explain the method/steps used.

In [ ]:
Tickers = ['GOOG']
date_1 = dt.date(2013, 1, 1)
date_2 = dt.date(2017, 12, 31)
price_data = yf.download(Tickers, date_1, date_2, interval = '1mo')

In [ ]:
daily_returns = price_data / price_data.shift(1) - 1.0
daily_returns.dropna(inplace = True)
daily_returns.drop(labels = ['Open', 'High', 'Low', 'Close', 'Volume'], axis = 1, inplace  = True)

In [ ]:
plt.figure()
plt.hist(daily_returns.values, bins = 2)
plt.show()

Therefore we can consider that the stock price will make an approximate monthly return of 0.20 with probability 0.20 and a return of -0.08 with probability 0.80.

In [ ]:
p = 0.2
q = 1 - p
u = 0.20
d = -0.08
binomial_tree = {}
S0 = price_data['Adj Close'].iloc[-1]
binomial_tree[str(0)] = [np.round(S0,2)]
monthly_expected_stock_price = {}
monthly_expected_stock_price[str(0)] = np.round(S0,2)
for i in range(6):
    temp = []
    exp_price = []
    for S in binomial_tree[str(i)]:
        S_u = S * (1+u)
        S_d = S * (1+d)
        temp.append(np.round(S_u,2))
        temp.append(np.round(S_d,2))
        exp_price.append(p*S_u + q*S_d)
    binomial_tree[str(i+1)] = temp
    monthly_expected_stock_price[str(i+1)] = np.round(mean(exp_price),2)
print(binomial_tree)

In [ ]:
count = 0
for S in binomial_tree['6']:
    if S > S0:
        count += 1
print('The probability that at the end of the 6 months the period price will be greater than the starting price is :', count/len(binomial_tree['6']))

In [ ]:
print(monthly_expected_stock_price)

# Question 3:

Consider an investment portfolio consisting of 200K in Aluminium and 500K in Zinc. The daily volatility of Aluminium is 0.70% and for Zinc is 0.20%. The correlation coefficient between them is 0.70.

i.Define Value at Risk and why it is applied. 

ii.Identifysome of its shortcomings and how other measures tend to address these. 

iii.Calculate the 15-day, 95% and 99% Value at Risk (VaR) of Aluminium and Zinc separately. 

iv.Calculate the 15-day, 95% and 99% Value at Risk (VaR) of the Aluminium and Zinc portfolio. 

v.Assuming an annualized rate return of 12% for Aluminium, use Geometric Brownian Motion to project the price of Aluminium over a year, at a monthly interval. 

Using the terminal prices at the end of the 1 year, calculate the 95% Value at Risk (VaR).  

In the simulation you are expected to project 500 possible paths.Make use of plots to explain your outcomes.

In [ ]:
pos_alu = 200000 # total investment position in Aluminum 
pos_zinc = 500000 # total investment position in Zinc
portfolio = pos_alu + pos_zinc # total portfolio position
vol_alu = 0.7/100 # daily volatility of aluminum
vol_zinc = 0.2/100 # daily volatility of zinc
corr = 0.7

In [ ]:
# confidence level at 95% is 1.96 so:
VaR_alu = (np.sqrt(15) * 1.96 * vol_alu) * pos_alu
VaR_zinc = (np.sqrt(15) * 1.96 * vol_zinc) * pos_zinc
print('15-day 95% VaR of Aluminum: ', VaR_alu)
print('15-day 95% VaR of Zinc: ', VaR_zinc)

In [ ]:
# confidence level at 99% is 2.576 so:
VaR_alu = (np.sqrt(15) * 2.576 * vol_alu) * pos_alu
VaR_zinc = (np.sqrt(15) * 2.576 * vol_zinc) * pos_zinc
print('15-day 99% VaR of Aluminum: ', VaR_alu)
print('15-day 99% VaR of Zinc: ', VaR_zinc)

In [ ]:
# confidence level at 95% is 1.96 so:
VaR_alu = (np.sqrt(15) * 1.96 * vol_alu) * pos_alu
VaR_zinc = (np.sqrt(15) * 1.96 * vol_zinc) * pos_zinc
print('15-day 95% VaR of the portfolio ', np.sqrt(VaR_alu + VaR_zinc + 2 * corr * VaR_alu * VaR_zinc))
# confidence level at 99% is 2.576 so:
VaR_alu = (np.sqrt(15) * 2.576 * vol_alu) * pos_alu
VaR_zinc = (np.sqrt(15) * 2.576 * vol_zinc) * pos_zinc
print('15-day 99% VaR of the portfolio ',  np.sqrt(VaR_alu + VaR_zinc + 2 * corr * VaR_alu * VaR_zinc))